# Spark Assignment  - Articles


#### I. Extract:  Load the data<br>
- Read data as json via spark dataframe

#### II.Transform: Exploratory data analysis using spark df<br>
- Unique Id count<br>
- Remove the html tags column “Article_Description” and “Full_Article”<br>
- Merge the columns “Heading”, “Article_Description” and “Full_Article” separated by space and place the merged text in a new column name “Preprocessed_Text”<br>
- select columns Id	Preprocessed_Text;	Article_Type;	Tonality;    Outlets<br>
- new column outlet_tags based on outlet text with .com as website and rest as App<br>
- show df<br>
- GroupBy Article_Type, Tonality and count<br>

#### III. Load: Save analysis report<br>
- show df, save as files(TXT)


In [18]:
# To install Spark
# !pip install spark

In [19]:
# Importing necessary libraries
import os
import spark
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession, types
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace, split, explode, struct, col, concat, lit, when
import pandas as pd

In [20]:
# Creating a spark session
spark = SparkSession.builder.master('local').appName('Json File').getOrCreate()

### Extract: Load the data

In [21]:
# Reading JSON file into a dataframe
df = spark.read.json("articles.json", multiLine=True)
df.printSchema()

root
 |-- Article.Banner.Image: string (nullable = true)
 |-- Article.Description: string (nullable = true)
 |-- Article_Type: string (nullable = true)
 |-- Full_Article: string (nullable = true)
 |-- Heading: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Outlets: string (nullable = true)
 |-- Tonality: string (nullable = true)



In [22]:
# To see the dataframe
df.show()

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|Article.Banner.Image| Article.Description|Article_Type|        Full_Article|             Heading|                  Id|             Outlets|Tonality|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|                    |<p>The helicopter...|  Commercial|<p>The helicopter...|A Puzzling Maneuv...|d6995462-5e87-453...|        Essex Caller|Negative|
|                    |<p>A year after t...|  Commercial|<p>A year after t...|Bell�s Nexus Air ...|8b05e939-a89e-454...|Aviation Week Net...|Positive|
|http://images.tmt...|<p>Bell released ...|  Commercial|<p>Bell released ...|Bell Helicopter S...|69fcd400-bceb-425...|             TMTPost|Positive|
|http://www.fredzo...|<p>Bell est une s...|  Commercial|<p>Bell est une s...|BELL D�VOILE LA C...|17

### Transform: Exploratory data analysis using spark df

In [23]:
## Q1. Unique ID count
print('Unique number of ids present: ',df.select('Id').distinct().count())

Unique number of ids present:  4305


In [24]:
# Changing the name of the column to avoid errors
df = df.withColumnRenamed('Article.Description','Article_Description')
df.printSchema()

root
 |-- Article.Banner.Image: string (nullable = true)
 |-- Article_Description: string (nullable = true)
 |-- Article_Type: string (nullable = true)
 |-- Full_Article: string (nullable = true)
 |-- Heading: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Outlets: string (nullable = true)
 |-- Tonality: string (nullable = true)



In [25]:
## Q2. Function to remove the html tags from column “Article_Description” and “Full_Article”

def remove_html(df,colname):
    df = df.withColumn(colname, regexp_replace(colname,"[<p></p>&nbs]",""))
    return df

df = remove_html(df,'Article_Description')
df = remove_html(df,'Full_Article')

# To see if the tags are removed
df.show()

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|Article.Banner.Image| Article_Description|Article_Type|        Full_Article|             Heading|                  Id|             Outlets|Tonality|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|                    |The helicoter tha...|  Commercial|The helicoter tha...|A Puzzling Maneuv...|d6995462-5e87-453...|        Essex Caller|Negative|
|                    |A year after teai...|  Commercial|A year after teai...|Bell�s Nexus Air ...|8b05e939-a89e-454...|Aviation Week Net...|Positive|
|http://images.tmt...|Bell releaed the ...|  Commercial|Bell releaed the ...|Bell Helicopter S...|69fcd400-bceb-425...|             TMTPost|Positive|
|http://www.fredzo...|Bell et ue ocieac...|  Commercial|Bell et ue ocieac...|BELL D�VOILE LA C...|17

In [26]:
## Q3. Merge the columns “Heading”, “Article_Description” and “Full_Article” separated by space and place the merged text in a new column name “Preprocessed_Text”

def col_combine(df, lst, colname):
    return df.withColumn(colname, concat(col(lst[0]), lit(" "), col(lst[1]), lit(" "), col(lst[2])))
lst_cols = ['Heading', 'Article_Description', 'Full_Article']
df = col_combine(df, lst_cols, 'Preprocessed_Text')
df.printSchema()

root
 |-- Article.Banner.Image: string (nullable = true)
 |-- Article_Description: string (nullable = true)
 |-- Article_Type: string (nullable = true)
 |-- Full_Article: string (nullable = true)
 |-- Heading: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Outlets: string (nullable = true)
 |-- Tonality: string (nullable = true)
 |-- Preprocessed_Text: string (nullable = true)



In [27]:
# Checking the first record to see the transformation
df.select('Preprocessed_Text').collect()[0]

Row(Preprocessed_Text='A Puzzling Maneuver, Then Freefall: NTSB Report Provides New Details in Southeast Alaska Helicopter Crash That Killed 3 The helicoter that crahed i Southeat Alaka i late Setemer, killig three eole, etered a 500-foot freefall efore droig to a Glacier Bay Natioal Park each, accordig to y the Natioal Traortatio Safety Board. The relimiary NTSB reort releaed Friday offer o official roale caue. That determiatio wolquo;t e made util ext year at the earliet. The helicoter that crahed i Southeat Alaka i late Setemer, killig three eole, etered a 500-foot freefall efore droig to a Glacier Bay Natioal Park each, accordig to y the Natioal Traortatio Safety Board.;The relimiary NTSB reort releaed Friday offer o official roale caue. That determiatio wolquo;t e made util ext year at the earliet.')

In [28]:
## Q4. Select columns :  Id, Preprocessed_Text, Article_Type, Tonality outlet
clean_df = df.select('Id', 'Preprocessed_Text', 'Article_Type', 'Tonality', 'Outlets')
clean_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Preprocessed_Text: string (nullable = true)
 |-- Article_Type: string (nullable = true)
 |-- Tonality: string (nullable = true)
 |-- Outlets: string (nullable = true)



In [29]:
## Q5. Creating new column outlet_tags based on outlet text with .com as website and rest as App
clean_df = clean_df.withColumn('outlet_tags', when(clean_df.Outlets.like('%.%'), \
                                        lit('website')).otherwise(lit('app')))

## Q6. Show Dataframe
clean_df.show()

+--------------------+--------------------+------------+--------+--------------------+-----------+
|                  Id|   Preprocessed_Text|Article_Type|Tonality|             Outlets|outlet_tags|
+--------------------+--------------------+------------+--------+--------------------+-----------+
|d6995462-5e87-453...|A Puzzling Maneuv...|  Commercial|Negative|        Essex Caller|        app|
|8b05e939-a89e-454...|Bell�s Nexus Air ...|  Commercial|Positive|Aviation Week Net...|        app|
|69fcd400-bceb-425...|Bell Helicopter S...|  Commercial|Positive|             TMTPost|        app|
|17943578-c11b-414...|BELL D�VOILE LA C...|  Commercial|Positive|            Fredzone|        app|
|f33c7b11-5f77-4a9...|Les premiers reto...|  Commercial|Positive|           FrenchWeb|        app|
|142dd70c-cf18-42d...|Highlights of CES...|  Commercial|Positive|      The Daily Star|        app|
|f096edd3-13db-4ae...|Le Concept Vision...|  Commercial|Positive|        Eric Houguet|        app|
|f8f917ec-

In [30]:
## Q7. GroupBy Article_Type, Tonality and count
file = clean_df.groupBy('Article_Type', 'Tonality').count()
file.show()

+------------------+--------+-----+
|      Article_Type|Tonality|count|
+------------------+--------+-----+
|          Training|Positive|    2|
|         Financing| Neutral|    1|
|          Military| Neutral|  178|
|          Military|        |  185|
|          Military|Negative|   53|
|Support & Services|Positive|   26|
|            Others|Positive|   45|
|            Others| Neutral|    7|
|          Training|        |    4|
|          Military|Positive| 1261|
|        Executives|        |    6|
|         Financing|        |    2|
|        Commercial| Neutral|   68|
|        Commercial|        |  235|
|         Financing|Positive|    6|
|        Commercial|Negative|  278|
|        Commercial|Positive| 1889|
|        Executives|Positive|   57|
|        Executives| Neutral|    2|
+------------------+--------+-----+



### Load: Save analysis report

In [31]:
file = file.toPandas()
file.to_csv('Output.txt', index=False)
print('Successfully saved at ',os.getcwd())

Successfully saved at  D:\M.Sc BDA\Coding\BD3P#\BD3P2_PySpark
